There is a very hacky way to overwrite some MPSKit methods with even hackier tightloop code. We can use this to explore the impact of the kind of caching that tightloop does in practical MPSKit code, without having to build an extensive fork.

I see a large speedup in some cases, but the tightloop mechanism is extremely tedious to use in practice. How close can we get with more extensive caching in TensorKit?

In [1]:
using Revise, CodeTracking, MPSKit, MPSKitExperimental, TensorKit, TensorOperations,MPSKitModels

In [2]:
# this might break if I start to tighthack kwdef functions? If not, this could be a fun package

detourtable = Any[]
for detour in methods(MPSKitExperimental.tighthack)
    orig = which(Tuple{detour.sig.parameters[2:end]...})
    orig_expr = definition(orig)
    push!(detourtable,(detour,orig,orig_expr))
end

function enable_highjack()
    
    tsymb = GlobalRef(MPSKitExperimental,:tighthack)

    for (detour,orig,orig_expr) in detourtable
        
        orsigparams = orig.sig.parameters

        pars = [gensym() for v in orsigparams]
        fdeff = Expr(:call,[Expr(:(::),a,b) for (a,b) in zip(pars,orsigparams)]...)
        body = Expr(:call,tsymb,pars...)
        MPSKit.eval(Expr(:function,fdeff,body))
    end
end

function disable_highjack()
    
    for (detour,orig,orig_expr) in detourtable
        MPSKit.eval(orig_expr)
    end
end

disable_highjack (generic function with 1 method)

# DMRG

In [10]:
fmps_len = 20;
middle_site = Int(round(fmps_len/2))

ts = FiniteMPS(rand,ComplexF64,fmps_len,Rep[SU₂](1=>1),Rep[SU₂](i => 20 for i in 0:20));
th_orig = heisenberg_XXX(SU2Irrep);

In [13]:
disable_highjack()
find_groundstate(ts,th_orig,DMRG(maxiter=10,verbosity=false));
@time find_groundstate(ts,th_orig,DMRG(maxiter=10));

┌ Info: DMRG init:	obj = +1.130573269071e+01	err = 1.2015e-01
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/dmrg.jl:34
┌ Info: DMRG   1:	obj = -2.683997864950e+01	err = 1.0291725359e-03	time = 7.22 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/dmrg.jl:56
┌ Info: DMRG   2:	obj = -2.683997865017e+01	err = 5.3663440204e-07	time = 1.91 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/dmrg.jl:56
┌ Info: DMRG   3:	obj = -2.683997865017e+01	err = 7.4542765139e-08	time = 3.31 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/dmrg.jl:56
┌ Info: DMRG   4:	obj = -2.683997865017e+01	err = 2.1688012451e-08	time = 2.82 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/dmrg.jl:56
┌ Info: DMRG   5:	obj = -2.683997865017e+01	err = 7.1748517293e-09	time = 2.77 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algor

 29.015868 seconds (77.35 M allocations: 13.537 GiB, 4.82% gc time, 0.44% compilation time: 100% of which was recompilation)


┌ Warning: DMRG cancel 10:	obj = -2.683997865017e+01	err = 1.0426042975e-09	time = 28.45 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/dmrg.jl:54


In [14]:
enable_highjack()
find_groundstate(ts,th_orig,DMRG(maxiter=10,verbosity=false));
@time find_groundstate(ts,th_orig,DMRG(maxiter=10));

┌ Info: DMRG init:	obj = +1.130573269071e+01	err = 1.2015e-01
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/dmrg.jl:34
┌ Info: DMRG   1:	obj = -2.683997864950e+01	err = 1.0291725305e-03	time = 3.35 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/dmrg.jl:56
┌ Info: DMRG   2:	obj = -2.683997865017e+01	err = 5.3663440634e-07	time = 1.51 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/dmrg.jl:56
┌ Info: DMRG   3:	obj = -2.683997865017e+01	err = 7.4542765255e-08	time = 2.05 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/dmrg.jl:56
┌ Info: DMRG   4:	obj = -2.683997865017e+01	err = 2.1688012636e-08	time = 2.01 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/dmrg.jl:56
┌ Info: DMRG   5:	obj = -2.683997865017e+01	err = 7.1748517119e-09	time = 1.85 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algor

 20.774627 seconds (49.43 M allocations: 11.131 GiB, 5.12% gc time, 0.62% compilation time: 100% of which was recompilation)


┌ Warning: DMRG cancel 10:	obj = -2.683997865017e+01	err = 1.0426048562e-09	time = 20.09 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/dmrg.jl:54


# VUMPS

In [3]:
ts = InfiniteMPS(rand,ComplexF64,[Rep[SU₂](1=>1)],[Rep[SU₂](i => 20 for i in 0:10)]);
th_orig = heisenberg_XXX(SU2Irrep);

In [9]:
disable_highjack()
find_groundstate(ts,th_orig,VUMPS(maxiter=10,verbosity=0));
@time find_groundstate(ts,th_orig,VUMPS(maxiter=10));

┌ Info: VUMPS init:	obj = +8.454997089671e-01	err = 2.8952e-01
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/vumps.jl:36
┌ Info: VUMPS   1:	obj = -3.547799826658e-01	err = 1.1617452974e-01	time = 1.21 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/vumps.jl:69
┌ Info: VUMPS   2:	obj = -3.984967800976e-01	err = 3.7320844819e-01	time = 1.75 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/vumps.jl:69
┌ Info: VUMPS   3:	obj = -1.141964884437e+00	err = 2.3090571709e-01	time = 1.58 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/vumps.jl:69
┌ Info: VUMPS   4:	obj = -8.257956504802e-01	err = 3.1400739517e-01	time = 2.38 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/vumps.jl:69
┌ Info: VUMPS   5:	obj = -1.105247485743e+00	err = 2.6477626629e-01	time = 2.89 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYo

 20.047041 seconds (76.84 M allocations: 15.389 GiB, 8.17% gc time, 0.03% compilation time)


┌ Warning: VUMPS cancel 10:	obj = -1.401484038671e+00	err = 1.0670130366e-05	time = 19.89 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/vumps.jl:67


In [8]:
enable_highjack()
find_groundstate(ts,th_orig,VUMPS(maxiter=10,verbosity=0));
@time find_groundstate(ts,th_orig,VUMPS(maxiter=10));

┌ Info: VUMPS init:	obj = +8.454997089671e-01	err = 2.8952e-01
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/vumps.jl:36
┌ Info: VUMPS   1:	obj = -3.547799837176e-01	err = 1.1617451192e-01	time = 0.68 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/vumps.jl:69
┌ Info: VUMPS   2:	obj = -3.985211623992e-01	err = 3.7320784435e-01	time = 0.94 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/vumps.jl:69
┌ Info: VUMPS   3:	obj = -1.133996351875e+00	err = 2.3237648081e-01	time = 0.90 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/vumps.jl:69
┌ Info: VUMPS   4:	obj = -8.025421989952e-01	err = 3.1933281492e-01	time = 1.33 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/vumps.jl:69
┌ Info: VUMPS   5:	obj = -7.671177726974e-01	err = 3.0447630261e-01	time = 1.56 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYo

 10.954443 seconds (7.42 M allocations: 8.422 GiB, 3.27% gc time, 0.04% compilation time)


┌ Warning: VUMPS cancel 10:	obj = -1.401480627914e+00	err = 8.6733924876e-04	time = 10.87 sec
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/vumps.jl:67
